In [0]:
from pyspark.sql.functions import col, to_date, when, current_date, datediff, first, sum as _sum, max as _max
from pyspark.sql.types import DoubleType

# 1. CSV 데이터 로드
df = spark.read.csv("/FileStore/tables/Walmart_customer_purchases.csv", header=True, inferSchema=True)

# 2. 데이터 스키마 확인 (실제 컬럼: Customer_ID, Purchase_Date, Purchase_Amount, Repeat_Customer, Category 등)
df.printSchema()

# 3. 컬럼 타입 변환
# - Purchase_Date를 날짜형으로 변환 (형식은 CSV에 맞게 조정: MM/dd/yyyy)
# - Purchase_Amount를 DoubleType으로 캐스팅
df = df.withColumn("Purchase_Date", to_date(col("Purchase_Date"), "MM/dd/yyyy")) \
       .withColumn("Purchase_Amount", col("Purchase_Amount").cast(DoubleType()))

# 4. Repeat_Customer 컬럼 활용: "Yes"이면 재구매이므로 frequency = 2, "No"이면 첫 구매이므로 1로 설정
df = df.withColumn("frequency", when(col("Repeat_Customer") == "Yes", 2).otherwise(1))

# 5. 고객별 RFM 지표 계산
# - recency: 고객별 가장 최근 구매일과 현재 날짜 간의 차이
# - frequency: 고객별 구매횟수는 CSV에 한 행씩 기록되었으므로, 각 행의 frequency 값 사용 (groupBy 후 first() 사용)
# - monetary: 고객별 총 구매 금액 (만약 고객별로 여러 행이 있을 경우 _sum, 한 행이면 그대로)
rfm_df = df.groupBy("Customer_ID").agg(
    datediff(current_date(), _max("Purchase_Date")).alias("recency"),
    first("frequency").alias("frequency"),
    _sum("Purchase_Amount").alias("monetary")
)

# 6. 평균 구매 금액 계산 (avg_purchase_amount = monetary / frequency)
rfm_df = rfm_df.withColumn("avg_purchase_amount", col("monetary") / col("frequency"))

# 7. 결과 확인
rfm_df.show(10, truncate=False)

# 8. Delta 테이블로 저장 (기존 데이터는 덮어쓰게 됨)
rfm_df.write.format("delta").mode("overwrite").option("overwriteSchema", "true").save("/delta/Walmart_RFM")

